In [1]:
import sys
sys.path.append('../../Scripts')
import pandas as pd
from DDBpy_auth import DDBAuth #https://github.com/arup-group/ddbpy_auth
import ddb_get_post as pyddb
import arupcomputepy
import json

from typing import List
import copy

Using default account: Ash.Kulkarni@arup.com


In [2]:
import ddb_client
import asyncio

Using default account: Ash.Kulkarni@arup.com


In [ ]:
def validate_inputs(parameter_ids:List[str], parameter_type_id:str, unit_id:str):
    # check the parameter types are correct?
    # check the units are correct?
    pass

In [ ]:
# calculate daily cold water consumption
def dwelling_daily_cw_consumption(project_id, asset_ids:List[str]):
    
    def dwelling_consumption(number_of_bedrooms:int):
        # number of bedrooms: daily demand (l) per bedroom - 3 indicates 3 or more bedrooms
        cw_consumption = {1:210,
                          2:130,
                          3:100}
        if number_of_bedrooms > 3:
            return number_of_bedrooms*cw_consumption[3]
        else:
            return number_of_bedrooms*cw_consumption[number_of_bedrooms]
    
    
    number_of_bedrooms_id = "5686f7ff-4678-4edf-8cfc-c20c9dcf24b1"
    daily_water_consumption_id = "a505e23f-0c0c-4722-be31-04304ef698a7"
    litres_id = "b4b05e4b-9a20-42ec-a508-16628d3ac1b1"
    industry_guidance_id = "e674c901-5844-4d64-8830-2933c14a4854"
    
    #validate_inputs()

    inputs = {x["parents"][0]["id"]:x["revision"]["values"][0]["value"] for x in pyddb.get_parameters(asset_id=asset_ids, parameter_type_id=number_of_bedrooms_id, page_limit=9999)}
    outputs = {k:dwelling_consumption(v) for (k,v) in inputs.items()}
    source_id = pyddb.add_new_source(
        project_id=project_id,
        source_type_id= industry_guidance_id,
        source_title= "Institute of Plumbing - Plumbing Engineering Services Design Guide",
        source_reference= "Table 2"
    )
    
    
    pyddb.upload_parameters(
        project_id = project_id,
        asset_ids = asset_ids,
        parameter_type_ids = [daily_water_consumption_id]*len(outputs),
        values= list(outputs.values()),
        unit_ids= [litres_id]*len(outputs),
        source_ids= source_id*len(outputs),
    )
    return


def subprocess_1(project_id, parent_ids:List[str]):
    """Enter group parent_ids [flats a, flats b]"""
    for asset_id in parent_ids:
        asset_ids = pyddb.get_descendants_of(asset_id = asset_id)["asset"]["children"]
        dwelling_daily_cw_consumption(project_id = project_id, asset_ids=asset_ids)

In [ ]:
# calculate daily cold water consumption
def building_daily_cw_consumption(project_id:str, flat_asset_ids: List[str], building_asset_id: str):
    #validate_inputs()
    daily_water_consumption_id = "a505e23f-0c0c-4722-be31-04304ef698a7"
    litres_id = "b4b05e4b-9a20-42ec-a508-16628d3ac1b1"

    parameters = pyddb.get_parameters(asset_id = flat_asset_ids, parameter_type_id = daily_water_consumption_id)
    total = sum(x["revision"]["values"][0]["value"] for x in parameters)
    
    
    derived_value_id = 'f266bbf1-0126-40b9-8fda-9f92a2731af9'
    source_id = pyddb.add_new_source(
        project_id = project_id,
        source_type_id = derived_value_id,
        source_title = "CWS Calc Process",
        source_reference = "Subprocess 2")
    
    pyddb.upload_parameters(
        project_id = project_id,
        asset_ids = [building_asset_id],
        parameter_type_ids = [daily_water_consumption_id],
        values = [total],
        unit_ids = [litres_id],
        source_ids = source_id
    )
    return

def subprocess_2(project_id:str, data: dict[str:List[str]]):
    
    for building_asset_id, flat_asset_ids in data.items():
        building_daily_cw_consumption(
            project_id = project_id,
            flat_asset_ids = flat_asset_ids,
            building_asset_id = building_asset_id
            )

In [ ]:
# calculate daily cold water consumption
def product_cw_storage_sizing(building_asset_ids: List[str]):
    # validate_inputs()
    daily_water_consumption_id = "a505e23f-0c0c-4722-be31-04304ef698a7"
    cold_water_diversity = "3cbc7bb3-3815-4c03-a2d6-361713626559"

    cubic_meter_id = "2ae7438a-4b8c-4f66-be4f-01315dc0cb8d"
    
    parameters = pyddb.get_parameters(
        asset_id=building_asset_ids,
        parameter_type_id=[daily_water_consumption_id, cold_water_diversity],
        page_limit=9999,
    )
    inputs = {
        x["parents"][0]["id"]: {
            x["parameter_type"]["name"]: x["revision"]["values"][0]["value"]
            for x in parameters
        }
        for x in parameters
    }
    
    for k,v in inputs.items():
        diversified_volume = float(v["Daily cold water consumption"]) * float(v["Domestic cold water diversity factor"])
        diversified_volume /= 1000
        
        # add system
        dcw_system_id = pyddb.add_new_asset(
            asset_type_id = "",
            project_id = project_id,
            asset_name = "Domestic Cold Water",
            parent_id = k
        )
        
        # add sub system
        dcw_sub_system_id = pyddb.add_new_asset(
            asset_type_id = "",
            project_id = project_id,
            asset_name = "Domestic Cold Water",
            parent_id = dcw_system_id
        )

        # add product type
        cold_water_tank_id = pyddb.add_new_asset(
            asset_type_id = "",
            project_id = project_id,
            asset_name = "Cold Water Storage Tank",
            parent_id = dcw_sub_system_id
        )
        
        pyddb.upload_parameters(project_id = project_id,
                               asset_ids = [cold_water_tank_id],
                               parameter_type_ids = [""],
                               values = [],
                               unit_ids = [],
                               source_ids = [])
        
    # get value of the parameters
    # sum values
    # post assets
    # post parameter to storage tank asset
    return diversified_volume

In [ ]:
pyddb.site = "sandbox"
product_cw_storage_sizing(["8be603c4-dbc9-480a-b3ec-e9cb86e5e190"])

In [ ]:
def main(project_number:str, group_ids:List[str]):
    # these "group_ids" are used because their children are inputs and parents are outputs, this is how it should relate for any project

    project_id = pyddb.get_project_id(project_number)
    
    subprocess_1(
        project_id=project_id,
        parent_ids= group_ids
    )

    subprocess_2(
        project_id=project_id,
        data =     {
            x["parent"]: x["children"]
            for x in [
                pyddb.get_descendants_of(asset_id=asset_id)["asset"]
                for asset_id in group_ids
            ]
        }
    )




In [ ]:
pyddb.site = "sandbox"
project_number = "11252300", 
    group_ids = [
        "43719f8d-ccfc-4ce5-bede-083030a46c1c",
        "5e045d75-858b-42f6-b428-e173ae00dfb4",
        "36a17f53-da00-4b10-b2d2-ad5d5cb94909",
        "1afe483c-ec99-4503-aa14-e04792c2018a",
        "e7cea058-b025-4ec5-a0bc-7d5b16711630"
    ]


main(
    project_number = project_number, 
    group_ids = group_ids
)


process:
    (sizing cw storage tank... description)
    
        can be run on asset type: building
    
        input item types:
            
            building + daily cw consumption
            building + cw diversity
            
            flat + number of bedrooms
    
    run for each building

    run for selected building(s)
    
    run for selected building(s) but deselect subprocesses

    run for building(s) that are missing the output parameters
    
    run for building(s) that have not had this run ever
    
    run for building(s) that have not had this run in the last 30 days - would you like to schedule this?
    
    see scheduled processes:
        
        process x is running on buildings [a, b, c] but skipping subprocesses [a.a.23, b.a.34] every 30 days

sub process 1:
    (find daily flat cw consumption by converting number of bedrooms to a dialy consumption using cibse guide g)
    
        ***can be run on zone type: flat/apartment - a project specific, group of a specific asset type with an arbitrarily named tag
    
        in the future, flats would be zones; a group of tagged assets
        
        input item types:
            flat + number of bedrooms

    run for single flat
    
    run for selected flats
    
    run for all flats
    
    *same options as process*
    
sub process 2:
    (find daily building cw consumption but summing the cw consumption of a *selected zone?*)
    
        can be run on asset type: building
    
        (in the future, flats would be zones; a group of tagged assets)
        
        input item types:
            flat + number of bedrooms
    
    *same options as process*
    
sub process 3:
    (calculate diversified cw storage volume using daily building consumption and cw diversity, posting to *selected output asset?* or by default creates a system/subsystem/product?)
    
        can be run on asset type: building
    
        (in the future, flats would be zones; a group of tagged assets)
        
        input item types:
            flat + number of bedrooms
    
    *same options as process*